# Load and run le modelz

This creates a new instance of the BERT model, loads the saved state dictionary, moves the model to the device, and sets it to eval mode.

Also initialises the BERT tokenizer, which will be used for tokenising the input text.

In [5]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load('bertStateDict.pt'))
model.to(device)
model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


To make predictions with new input text, you can tokenise the text using BERT tokeniser and pass it through the loaded model.

Function takes input text, tokenises it, and passes the tokenised input through the loaded model.
Returns the predicted class and probability of input being toxic.

In [6]:
def predict_toxicity(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1)
        
        toxicity_score = probabilities[0][1].item() * 100
        
    return predicted_class.item(), toxicity_score

NOW USE IT ON SOME INPUT TEXT!

In [7]:
text = "I hate you"
predicted_class, toxicity_score = predict_toxicity(text)
print(f'Predicted class: {predicted_class}, Toxicity score: {toxicity_score:.2f}%')

Predicted class: 1, Toxicity score: 83.69%


In [15]:
text = ""
predicted_class, toxicity_score = predict_toxicity(text)
print(f'Predicted class: {predicted_class}, Toxicity score: {toxicity_score:.2f}%')

Predicted class: 0, Toxicity score: 5.32%
